In [ ]:
df_sliced_dict = {}
import pandas as pd
import numpy as np
from pandas import set_option
from collections import Counter
from numpy import mean
from numpy import std
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import confusion_matrix
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')

df=pd.read_csv('Results_9999_raw_avec_distances_phonologiques_for_ML.csv' , sep=',' , header=None)
df.columns = df.iloc[0]
df=pd.DataFrame(df)
df=df.drop(df.index[0])
df[['s_125', 's_250', 's_500', 's_750', 's_1000', 's_1500', 's_2000',
       's_3000', 's_4000', 's_6000', 's_8000', 's_10000']] = df[['s_125', 's_250', 's_500', 's_750', 's_1000', 's_1500', 's_2000',
       's_3000', 's_4000', 's_6000', 's_8000', 's_10000']].apply(pd.to_numeric)
df
for phoneme in df['phoneme_cible'].unique():
    print("\n###################################################################")
   
    print("\nDF pour le phoneme cible " + phoneme + ":")
    df_sliced_temp = pd.DataFrame()
    df_sliced_temp = df[df['phoneme_cible'] == phoneme]
    sorted_df=df_sliced_temp
    sorted_df=sorted_df.iloc[:, [1,2,3,4,5,6,7,8,9,10,11,12,13,18]]
    #print(sorted_df)
    #print(Counter(sorted_df['phoneme_cible_et_reconnu']))
    lst=sorted_df.groupby(['phoneme_cible_et_reconnu']).size()
    #print(lst)
    lst.to_csv('lst.csv') 
    df1 = pd.read_csv('lst.csv' , sep=',' , header=None)
    df1.columns = df1.iloc[0]
    df1=pd.DataFrame(df1)
    df1=df1.drop(df1.index[0])
    df1.columns=['phoneme_cible_et_reconnu','count']
    #print(df1)
    df_filter=df1.loc[(df1['count'] >50)]
    #print(df_filter)
    sorted_df=sorted_df[sorted_df.set_index(['phoneme_cible_et_reconnu']).index.isin(df_filter.set_index(['phoneme_cible_et_reconnu']).index)]
    print("\n")
    print(Counter(sorted_df['phoneme_cible_et_reconnu']))
    
    #### Exploratory data analysis 
    set_option('precision', 3)
    print("\n")
    print(sorted_df.describe())
    target = sorted_df.values[:,-1]
    counter = Counter(target)
    for k,v in counter.items():
        per = v / len(target) * 100
        print('Class=%s, Count=%d, Percentage=%.3f%%' % (k, v, per))
   
    ##### TRAINING ET EVALUATION
    print("\n\n############## TRAINING ET EVALUATION POUR LE PHONEME [" + phoneme + "]\n\n")
    data =sorted_df
    print(data)
    data = data.values
    X = data[:, :-1]
    y = data[:, -1]
    ####
    print("\n\n ######### predire le class le plus frequente ")
    def evaluate_model(X, y, model):
        # define evaluation procedure
        cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
        # evaluate model
        scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
        return scores
    model = DummyClassifier(strategy='most_frequent')
    scores = evaluate_model(X, y, model)
    # summarize performance
    print('Mean Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))
   
    ##### test train stratified simple
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)
    print("\n\n ######### linear discrimnant analysis")
    LDA=LinearDiscriminantAnalysis().fit(X_train, y_train)    
    LR= LogisticRegression(solver='lbfgs', multi_class='multinomial').fit(X_train, y_train)
    SVM=SVC(kernel='rbf', C=1).fit(X_train, y_train)
    # predict the test data
    y_pred_LDA= LDA.predict(X_test)
    #importing confusion matrix 
    confusion_LDA = confusion_matrix(y_test, y_pred_LDA)
    print('Confusion Matrix\n')
    print(confusion_LDA)
    #importing accuracy_score, precision_score, recall_score, f1_score
    print('\nAccuracy: {:.2f}\n'.format(accuracy_score(y_test, y_pred_LDA)))
    print('\nClassification Report\n')
    print(classification_report(y_test, y_pred_LDA))
    #####
    print("\n\n ######### regression logistic multinominal")
    y_pred_LR= LR.predict(X_test)
    confusion_LR = confusion_matrix(y_test, y_pred_LR)
    print('Confusion Matrix\n')
    print(confusion_LDA)
    print('\nAccuracy: {:.2f}\n'.format(accuracy_score(y_test, y_pred_LR)))
    print('\nClassification Report\n')
    print(classification_report(y_test, y_pred_LR))
    #####
    print("\n\n ######### support vector machine")
    y_pred_SVM= SVM.predict(X_test)
    confusion_SVM = confusion_matrix(y_test, y_pred_SVM)
    print('Confusion Matrix\n')
    print(confusion_SVM)
    print('\nAccuracy: {:.2f}\n'.format(accuracy_score(y_test, y_pred_SVM)))
    print('\nClassification Report\n')
    print(classification_report(y_test, y_pred_SVM))
    #####
    print("\n\n ######### under-over sampling")
    smp=SMOTE()
    #smp= RandomUnderSampler()
    Xsmp_train, ysmp_train = SMT.fit_resample(X_train, y_train)
    LDA_smp= LinearDiscriminantAnalysis().fit(Xsmp_train, ysmp_train)
    y_pred_LDA_smp= LDA_smp.predict(X_test)
    #importing confusion matrix 
    confusion_LDA_smp = confusion_matrix(y_test, y_pred_LDA_smp)
    print('Confusion Matrix\n')
    print(confusion_LDA_smp)
    #importing accuracy_score, precision_score, recall_score, f1_score
    print('\nAccuracy: {:.2f}\n'.format(accuracy_score(y_test, y_pred_LDA_smp)))
    print('\nClassification Report\n')
    print(classification_report(y_test, y_pred_LDA_smp))